#### For overrride patches, we use checklists to evaluate how well our model can:
- use patches when they refer to abstract conditions
- avoid spurious behaviors like matching based on keywords, ignoring negated contexts etc.

In [18]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from eval_utils import load_model

# TODO: make this notebook better. try to use mtl vs non mtl and see difference. 
# TODO: see what the baseline without any explanations or anything gets. 

#path_name = '/u/scr/smurty/LanguageExplanations/gpt2-finetune-generalization/models/t5-large-sst-no-exp'
#path_name = '/u/scr/smurty/LanguageExplanations/all_models/t5-large-sst-fix-mtl-4'


path_name='t5-sst-overrides-mtl-newest'
model_obj = load_model(path_name, primary_mode='exp_applies_predictor')

Some weights of T5ForConditionalGenerationMultipleHeads were not initialized from the model checkpoint at t5-large and are newly initialized: ['encoder.embed_tokens.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


primary mode: exp_applies_predictor
splicing parts from pretrained model


Some weights of T5ForConditionalGeneration were not initialized from the model checkpoint at t5-large and are newly initialized: ['encoder.embed_tokens.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
from data_fns import keyword_matching_real_words, keyword_matching_gibberish_words, aspect_abstraction_test_fn

all_datasets = {'keyword_real': keyword_matching_real_words(),
               'aspect': aspect_abstraction_test_fn(),
               'keyword_gibberish': keyword_matching_gibberish_words()}



### construct a label according to the conditions, since we have an override.
def cond2label_dict(cond, orig_label):
    is_food = 'food' in cond
    is_good = 'good' in cond
    label_name2label = {'positive': 1, 'negative': 0, 'NAN': -1}
    if is_food:
        dict_to_use = [label for label in orig_label if label['category'] == 'food'][0]
    else:
        dict_to_use = [label for label in orig_label if label['category'] == 'service'][0]
        
    label = label_name2label[dict_to_use['polarity']]
    cond_label = int(label_name2label[dict_to_use['polarity']] == is_good)
    return label, cond_label


def conds_and_labels(data_tuple):
    def helper(explanation, inputs):
        word = explanation.split(",")[0].split(" ")[-1]
        ### If word appears in label, then the explanation applies
        labels =  [int(word in cinput) for cinput in inputs]
        cond = " ".join(explanation.split(",")[0].split(" ")[1:])
        return cond, labels    

    data_conds = []
    data_label_sets = {}
    for explanation in data_tuple[-1]:
        cond, labels = helper(explanation, data_tuple[0])
        if 'positive' in explanation:
            l = [0, 1]
        else:
            l = [1, 0]
        data_conds.append((cond, l))
        data_label_sets[cond] = labels    
    return data_conds, data_label_sets


all_conds = {}
all_label_sets = {}
for key in all_datasets:
    val = all_datasets[key]
    if 'checklist' in key :
        conds = [('food is good', [0,1]), ('service is good', [0,1]), ('food is bad',[1,0]), ('service is bad',[1,0])]
        label_sets = {cond: [cond2label_dict(cond, l) for l in val[1]] for cond, _ in conds}
    else:
        conds, label_sets = conds_and_labels(val)
    
    all_conds[key] = conds
    all_label_sets[key] = label_sets

In [11]:
from eval_utils import predict_stuff
import numpy as np
import itertools


def apply_patch_soft(exp_applies_probs, baseline_probs, label_clause):    
    x = np.array([label_clause]).repeat(len(baseline_probs), 0)
    #print(x.shape)    
    applies_prob = exp_applies_probs[:, 1].reshape(-1, 1)
    #print(applies_prob)
    return applies_prob * x + (1 - applies_prob) * baseline_probs


def get_scores_multiple_patches_hard(data, cond_list, examine=False):
    no_exps = [('', ex) for ex in data[0]]
    no_exp_probs = predict_stuff(no_exps, [0]*len(no_exps), model_obj, 'p1', verbose=False, mode='task_predictor')
    cond_probs = []
    interpret_probs = []
    all_patched_probs = []
    for idx, (cond, label_clause) in enumerate(cond_list):
        if cond == '':
            continue
        contextualized = [(cond, ex) for ex in data[0]]
        output_probs = predict_stuff(contextualized, itertools.repeat(0), model_obj, 'p1', verbose=False)
        cond_probs.append(np.log(output_probs[:, 1])) # log(p(c | x))
        
        patched_probs = apply_patch_soft(output_probs, no_exp_probs, label_clause) #Pr(y | x, lp)        
        all_patched_probs.append(patched_probs[:, 1])
    # how much should each be weighted by?    
    # pick best patch and apply it! 
    all_patched_probs = np.stack(all_patched_probs, axis=1) # D x P
    cond_probs = np.stack(cond_probs, axis=1) # D x P
    best_patches = np.argmax(cond_probs, axis=1) # D x l
    
    ptrue = np.array([p[idx] for p, idx in zip(all_patched_probs, best_patches)])
    pfalse = 1.0 - ptrue
    return no_exp_probs, np.stack([pfalse, ptrue]).T

In [5]:
from collections import Counter
all_outputs = {}

for key in all_datasets:
    val = all_datasets[key]    
    all_cond = all_conds[key]
    label_sets = all_label_sets[key]
    baseline, patched = get_scores_multiple_patches_hard(val, all_cond)    
    print(np.mean(baseline.argmax(axis=1)==val[1]))
    print(np.mean(patched.argmax(axis=1)==val[1]))
 

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

...

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

0.49019607843137253
1.0


#### Next, we measure when patches are not relevant, how often does the model's behavior remain unchanged.

In [15]:
invariance_datasets = {'aspect_negated': aspect_abstraction_test_fn_negated()}

for key in invariance_datasets:
    val = invariance_datasets[key]    
    conds, label_sets = conds_and_labels(val)
    print(conds)
    
    baseline, patched = get_scores_multiple_patches_hard(val, conds)
    print(np.mean(baseline.argmax(axis=1)==patched.argmax(axis=1)))



[('food is described as weird', [1, 0]), ('food is described as surprising', [1, 0]), ('food is described as unexpected', [1, 0]), ('food is described as unusual', [1, 0])]


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

1.0


In [23]:
from data_fns import get_checklist_data_negated
data_curr = get_checklist_data_negated()

for key in data_curr:
    if 'good' in key:
        cons = [0, 1]
    else:
        cons = [1, 0]
    examples = [ex for t in data_curr[key] for ex in t['data']]
    baseline, patched = get_scores_multiple_patches_hard([examples, itertools.repeat(0)], [(key, cons)])
    print(np.mean(baseline.argmax(axis=1)==patched.argmax(axis=1)))



  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

1.0


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

1.0
